# `1.6 Microcontroller 🎛`

## <span style="color:tomato"> Overview </span>
- The AVR128DB64T is a 8-bit microcontroller developed by Microchip Technology which is based on the AVR 8-bit RISC architecture and is commonly used in embedded systems and microcontroller-based applications. The microcontroller is the brain of the LunaSat - it sends commands to other components and stores uploaded code.
---

## <span style="color:tomato"> Configurations </span>
The AVR128DB64T has dozens of modes across its peripherals. We will focus on the main modes that will have the greatest impact on the power usage of the microcontroller. It's important to note that the peripherals do draw power when active, but because there are more than 15 peripherals with many modes each, we do not have a strong understanding yet of what peripherals we need and in what modes we need them in. For now, we will conservatively estimate that the peripherals will draw <span style="color:orange">4 mA</span> in current. Since the microcontroller is essential to all mission operations and should operate in the same mode in certain scenarios across all LunaSats, your choice of microcontroller configurations will ultimately not be includded in the final set of configurations. However, this notebook will ultimately show you GLEE's process of determining how to configure the microcontroller so that its power usage is minimized while still providing an acceptable code execution speed. This tradeoff will become more clear once testing is done to measure code execution speed with different <span style="color:orange">CPU</span> clocks and frequencies.

- <span style="color:orange"> **CPU (Central Processing Unit)**</span>: In order to execute instructions (code), the CPU must be in Active mode. It can be put into sleep modes at certain times and woken with interrupt signals. Power consumption in these modes is dependent on what oscillator is used as a clock for the CPU and what the frequency of the oscillator is.
    - <span style="color:orange">Active</span>: All peripherals are running. See page 556 of the datasheet to find the values that correspond to the power consumption of different oscillators and frequencies.
    - <span style="color:orange">Idle</span>: Code execution is paused. All peripherals are running. All interrupt sources can wake the device.
    - <span style="color:orange">Standby</span>: Peripherals can be enabled or disabled. 
    - <span style="color:orange">Power-Down</span>: The Watchdog Timer (WDT) and the Periodic Interrupt Timer (PIT) are active. The only wake-up sources are the pin change interrupt, TWI address match, and CCL (if filter and edgedetect are disabled).
- <span style="color:orange"> **CLKCTRL (Clock Controller)**</span>: This peripheral is responsible for allowing users to select a particular oscillator that will be used as the Main Clock (CLK_MAIN) for the AVR128DB64T microcontroller.
    - <span style="color:orange">Internal High-Frequency Oscillator (OSCHF)</span>: The OSCHF supports output frequencies of 1, 2, 3, 4 MHz, and multiples of 4, up to 24 MHz.
    - <span style="color:orange">32.768 kHz Oscillator (OSC32K)</span>: The 32.768 kHz oscillator is optimized for Ultra Low-Power (ULP) operation. Power consumption is decreased at the cost of decreased accuracy compared to an external crystal oscillator.
    - <span style="color:orange">32.768 kHz Crystal Oscillator (XOSC32K)</span>: This clock is like the one above, but is instead external rather than internal.
    - <span style="color:orange">External Clock (EXTCLK)</span>: The EXTCLK is taken directly from the pin. This GPIO pin is automatically configured for the EXTCLK if any peripheral requests this clock. The maximum input frequency for the EXTCLK is 24 MHz.

## <span style="color:tomato"> Creating Configurations </span>

From the following options, create several configurations by choosing the CPU mode, the clock being used for CLK_Main, and the clock frequency. Low Power is an option specifically for the XOSC32K clock. The Prescalar Divisor is a number that allows for the changing of the clock frequency during runtime. It can only reduce the frequency, rather than raise it. The frequency of the clock changes the power usage of the microcontroller while also determining how fast code can be executed. The faster the frequency, the faster code executes and the more power is consumed.

- <span style="color:orange">**Modes**</span>: {"ACTIVE", "IDLE", "STANDBY", "POWER_DOWN"}
- <span style="color:orange">**Clock**</span>: {"OSCHF", "OSC32K", "XOSC32K", "EXTCLK"}
- <span style="color:orange">**Frequency**</span>: The range of possible values is dependent on the choice of clock.
    - "OSCHF": {1, 2, 3, 4, 8, 12, 16, 20, 24} MHz
    - "OSC32K": N/A. Set to 32.768 kHz.
    - "XOSC32K": N/A. Set to 32.768 kHz.
    - "EXTCLK": N/A. Determined by clock source. Capped at 24 MHz.
- <span style="color:orange">**Low Power**</span>: {"ON", "OFF"}
    - Can be used to lower the power usage of XOSC32K only. Not utilized for any other clock source.
- <span style="color:orange">**Prescalar Divisor**</span>: {1, 2, 4, 8, 16, 32, 64, 6, 10, 12, 24, 48}
    - 1 indicates that the clock frequency will not be divided and will not change. This parameter is important for clocks without varying maximum frequencies or for when tweaks want to be made without causing significant downtime while switching between configurations.
    
Certain configurations do not utilize all available settings. Below are the configuration formats for the 4 possible modes you can choose from. Anything with {} braces indicates a parameter you can change.

- <span style="color:orange">({ Mode }, "OSCHF", { Frequency }, "OFF", { Prescalar Divisor})</span>
- <span style="color:orange">({ Mode }, "OSC32K", 32.768, "OFF", { Prescalar Divisor})</span>
- <span style="color:orange">({ Mode }, "XOSC32K", 32.768, { Low Power }, { Prescalar Divisor})</span>
- <span style="color:orange">({ Mode }, "EXTCLK", { Frequency }, "OFF", { Prescalar Divisor})</span>

You should also choose how long the sensor will spend in that configuration (duration, in seconds). 
- <span style="color:orange">**Configuration duration (s)**</span>: {n} 

<span style="color:#18BF7D">For the following code, replace 'None' with appropriate values based on what you have learned above. Any lines without a 'None' should not be modified. Changing the values of the variables allows you to adjust a configuration consisting of a collection of these variables. At the end of the cell, these configurations, in addition to their durations and the sampling rates, are put together in a list.</span>

In [ ]:
import micropip
await micropip.install("matplotlib")
import matplotlib.pyplot as plt
await micropip.install("numpy")
import numpy as np
from source.AVR128DB64T import AVR128DB64T

# Example parameters

mode = "ACTIVE"
clock = "OSCHF"
freq = 4 #MHz
low_power = "OFF"
divisor = 1

example_configuration = (mode, clock, freq, low_power)
duration = 60

# Try creating 3 distinct configurations below by replacing None with appropriate values

# Configuration 1

mode = None
clock = None
freq = None #MHz
low_power = None
divisor = None

configuration_1 = (mode, clock, freq, low_power, divisor)
duration_1 = None

# Configuration 2

mode = None
clock = None
freq = None #MHz
low_power = None
divisor = None

configuration_2 = (mode, clock, freq, low_power, divisor)
duration_2 = None

# Configuration 3

mode = None
clock = None
freq = None #MHz
low_power = None
divisor = None

configuration_3 = (mode, clock, freq, low_power, divisor)
duration_3 = None


# Below, we group together the configuration tuple with a duration and sampling rate in another tuple
# These tuples are added to a list called modes_MCR
modes_MCR = [(configuration_1, duration_1, 0), (configuration_2, duration_2, 0), (configuration_3, duration_3, 0)]

## <span style="color:tomato"> Calculating Power Usage </span>

We can calculate the power usage based on the configuration to get an idea of how it's affected by the various settings. Power is measured in milliWatts, where 1 milliWatt is equivalent to 0.001 Joules of energy per second. 

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
power_1 = AVR128DB64T.compute_power(AVR128DB64T,*configuration_1)
power_2 = AVR128DB64T.compute_power(AVR128DB64T,*configuration_2)
power_3 = AVR128DB64T.compute_power(AVR128DB64T,*configuration_3)

print("Config. 1: ", power_1, "milliWatts")
print("Config. 2: ", power_2, "milliWatts")
print("Config. 3: ", power_3, "milliWatts")

plt.bar(1, power_1, label='Config. 1')
plt.bar(2, power_2, label='Config. 2')
plt.bar(3, power_3, label='Config. 3')

plt.xticks([])
plt.ylabel('Power in mW')
plt.grid()
plt.legend()
plt.show()

## <span style="color:tomato"> Configurations for a Lunar Mission </span>

The ideal modes and clock sources will highly dependent on the finalized mission timeline once further research and testing with the LunaSats is completed. Assume for now that while your configurations are running, the microcontroller must always be in active mode. A low frequency may result in it being more difficult to meet chosen sensor sampling rates, so consider using an internal clock (like OSCHF or OSC32K ) with a frequency / prescalar divisor that results in a balanced trade-off between speed and power usage. A lower power usage corresponds to slower code execution while a higher power usage corresponds to faster code execution.

<span style="color:#18BF7D">With these limitations in mind, revisit the configurations you set earlier and revise them so that they are well-suited for a lunar mission. </span>

## <span style="color:tomato"> Putting It All Together </span>

Finally, we'll take a look at the power and data usage of all of our configurations together. The time_step is the distance between data values in the plot, and the duration is how long the sensor is active for in the model. You can change them if you'd like. The duration must be at least as long as the duration of the first configuration.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
time_step = 1
total_duration = duration_1 + duration_2 + duration_3

mcr = AVR128DB64T(time_step = time_step, duration = total_duration, modes_MCR = modes_MCR, loop_rate = 20)
mcr_power, mcr_data, mcr_time = mcr.run_sim()

We'll store our generated data for use in the combined power and data model.

In [ ]:
%store mcr_power
%store mcr_data
%store mcr_time

[Click here to continue to the RF notebook.](1.7%20RF.ipynb)

If you would like to look at more detailed information for the AVR microcontroller, consider looking at the [AVR128DB64T Datasheet](http://ww1.microchip.com/downloads/en/DeviceDoc/AVR128DB28-32-48-64-DataSheet-DS40002247A.pdf).